且 IFRS 制度有規定一年要出 5 次財務報表，除了一年4次的季報表，還有年度報表，且都是採母子公司合併的合併報表。

每個制度都有它的好與壞，但 IFRS 改良了舊制度的一些缺點，卻也有它自己的不足，但市場對 IFRS 制度的高度採用原因是

**它使公司的比較更有公平性、提升對專業（或是投資上）數據判斷的精準度、更加透明…等等。**

過去 GAAP 制度，如果有一筆平常不會發生的損益，像是匯兌損益，可以列到非常損益項目。

IFRS 禁止使用非常損益，投資人需要從附錄中找尋重大損益的資料

In [1]:
import requests
import pandas as pd


In [2]:
url = 'https://mops.twse.com.tw/mops/web/t05st10_ifrs'

In [52]:
payload = {
       'encodeURIComponent': '1',
       'step': '1',
       'firstin': '1',
       'off': '1',
       'queryName': 'co_id',
       'inpuType': 'co_id',
       'TYPEK': 'all',
       'isnew': 'false',
       'co_id': '2457',
       'year': '111',
       'month': '06',
   }
res = requests.post(url, data = payload)
pd.read_html(res.text)[1]

,0,1
0,NaN,營收除權息電子書法說會庫藏股董監持股獨立董事董監酬金ETFTDR


In [53]:
temp=pd.read_html(res.text)[9]
temp2=pd.read_html(res.text)[10]

In [55]:
temp2

,項目,營業收入淨額
0,本月,1407257.00
1,去年同期,971600.00
2,增減金額,435657.00
3,增減百分比,44.84
4,本年累計,6361076.00
5,去年累計,5365959.00
6,增減金額,995117.00
7,增減百分比,18.54


#  取得單次月營收

In [61]:
def getMoM(sym, year, month): 
    month=str(month)
    url = 'https://mops.twse.com.tw/mops/web/t05st10_ifrs'
    payload = {
               'encodeURIComponent': '1',
               'step': '1',
               'firstin': '1',
               'off': '1',
               'queryName': 'co_id',
               'inpuType': 'co_id',
               'TYPEK': 'all',
               'isnew': 'false',
               'co_id': str(sym),
               'year': str(year),
               'month': '0'+month,
           }
    res = requests.post(url , data = payload)
    df = pd.read_html(res.text)[10]
    return df

# 沒有MoM的 

In [63]:
import time
# df=get_income_statement(2330,110,2)
def saveDataToExcel(stockID,start,end):
    interval=end-start+1 
    mySheet=[]
    df=[]
    counter=0
    with pd.ExcelWriter(f'{stockID}MoM.xlsx', engine='xlsxwriter') as writer:
        for i in range(interval):
            for j in range(12):
                try:
                    df.append(getMoM(stockID, end-i, 12-j))
                    mySheet.append(f'民國第{end-i}年{12-j}月')
                    df[counter].to_excel(writer, sheet_name=mySheet[counter])
                    counter=counter+1
                    time.sleep(10)
                except:
                    print(f'民國第{end-i}年{12-j}月 no data')
    counter=0
    with pd.ExcelWriter(f'{stockID}MoM.xlsx', engine='xlsxwriter') as writer:
        for subSheet in mySheet: 
                df[counter].to_excel(writer, sheet_name=subSheet, index=True, na_rep='NaN')
                # Auto-adjust columns' width
                for column in df[counter]:
                    column_width = 40
                    if(counter==0):
                        column_width=10
                    col_idx = df[counter].columns.get_loc(column)
                    writer.sheets[subSheet].set_column(col_idx, col_idx, column_width)
                counter=counter+1
    #     writer.save()
    print("File loading is finsh!")
saveDataToExcel(2457,111,111)

民國第111年12月 no data
民國第111年11月 no data
民國第111年10月 no data
民國第111年9月 no data
民國第111年8月 no data
民國第111年7月 no data
File loading is finsh!


In [110]:
# getMoM(2457, 111, 6)

In [64]:
temp2

,項目,營業收入淨額
0,本月,1407257.00
1,去年同期,971600.00
2,增減金額,435657.00
3,增減百分比,44.84
4,本年累計,6361076.00
5,去年累計,5365959.00
6,增減金額,995117.00
7,增減百分比,18.54


In [65]:
new_row = {'項目':'MoM', '營業收入淨額':100}
temp2 = temp2.append(new_row, ignore_index=True)

In [66]:
temp2

,項目,營業收入淨額
0,本月,1407257.00
1,去年同期,971600.00
2,增減金額,435657.00
3,增減百分比,44.84
4,本年累計,6361076.00
5,去年累計,5365959.00
6,增減金額,995117.00
7,增減百分比,18.54
8,MoM,100.00


# 加入MoM (這個月/上個月-1)*100 

In [108]:
import time
# df=get_income_statement(2330,110,2)
def saveDataToExcel(stockID,start,end):
    interval=end-start+1 
    mySheet=[]
    df=[]
    counter=0
    for i in range(interval):
        for j in range(12):
            try:
                df.append(getMoM(stockID, end-i, 12-j))
                mySheet.append(f'民國第{end-i}年{12-j}月')
                counter=counter+1
                time.sleep(10)
            except:
                print(f'民國第{end-i}年{12-j}月 no data')
    for j in range(counter-1):
        try:
            new_row = {'項目':'MoM', '營業收入淨額':(df[j].loc[0][1]/df[j+1].loc[0][1]-1)*100}
            print(new_row)
            print('previous',df[j].loc[0][1])
            df[j] = df[j].append(new_row, ignore_index=True)
        except:
            print(f'insert error')
    with pd.ExcelWriter(f'{stockID}MoM.xlsx', engine='xlsxwriter') as writer:
        for j in range(counter-1): 
            try:
                df[j].to_excel(writer, sheet_name=mySheet[j])
            except:
                print(f'excel error')
    counter=0
    with pd.ExcelWriter(f'{stockID}MoM.xlsx', engine='xlsxwriter') as writer:
        for subSheet in mySheet: 
                df[counter].to_excel(writer, sheet_name=subSheet, index=True, na_rep='NaN')
                # Auto-adjust columns' width
                for column in df[counter]:
                    column_width = 40
                    col_idx = df[counter].columns.get_loc(column)
                    writer.sheets[subSheet].set_column(col_idx, col_idx, column_width)
                counter=counter+1
    #     writer.save()
    print("File loading is finsh!")
    return df
res=saveDataToExcel(2457,111,111)

民國第111年6月 no data
民國第111年5月 no data
民國第111年4月 no data
民國第111年3月 no data
民國第111年2月 no data
民國第111年1月 no data
{'項目': 'MoM', '營業收入淨額': 42.971932024029535}
previous 1407257.0
{'項目': 'MoM', '營業收入淨額': -1.6125211286048602}
previous 984289.0
{'項目': 'MoM', '營業收入淨額': -11.118090340647601}
previous 1000421.0
{'項目': 'MoM', '營業收入淨額': 55.43988929919874}
previous 1125562.0
{'項目': 'MoM', '營業收入淨額': -35.31421710812528}
previous 724114.0
File loading is finsh!


In [109]:
res[0]

,項目,營業收入淨額
0,本月,1.407257e+06
1,去年同期,9.716000e+05
2,增減金額,4.356570e+05
3,增減百分比,4.484000e+01
4,本年累計,6.361076e+06
5,去年累計,5.365959e+06
6,增減金額,9.951170e+05
7,增減百分比,1.854000e+01
8,MoM,4.297193e+01
